In [81]:
import glob
import re
import itertools
import tqdm
import pandas as pd

In [82]:
df = pd.read_excel("../query/query copy.ods", engine="odf")
df

,Target,Query,Near,Near.1
0,1.1,poverty,eradicat\w*|reduc\w*|end|ending|alleviat\w*,NaN
1,1.1,poverty line\w*,NaN,NaN
2,1.1,poverty indicator,NaN,NaN
3,1.1,poverty,inequalit\w*,NaN
4,1.1,income,inequalit\w*,NaN
5,1.1,poverty,chronic\w*|extreme,NaN
6,1.2,poverty,living|life|child\w*|socioeconomic\w*|socio-ec...,NaN
7,1.3,social protection|economic marginalization|eco...,poverty|income,NaN
8,1.4,access|right\w*,economic resource\w*| basic service\w*,NaN
9,1.4,ownership\w*,land,NaN


In [83]:
def fix_length(lst, length):
    return (lst + [None] * length)[:length]

In [84]:
from pyexcel_ods import get_data
import json
query_file = "../query/query copy.ods"
excel_file = get_data(query_file)
lst = []
for sdg_number, sdg  in excel_file.items():
    for ind, line in enumerate(sdg):
        line = fix_length(line, 5)
        if ind == 0:
            continue
        # target, main_query, near_1, near_2, not_1 = line
        target, *query, not_1 = line
        query = [elem for elem in query if elem is not None]


        for chunk in itertools.permutations(query, len(query)):
            chunk = ["("+str(elem)+")" for elem in chunk]
            new_line = [sdg_number, target, '\W+(?:\w+\W+){0,3}?'.join(chunk), not_1]
            lst.append(new_line)


df_query = pd.DataFrame(data=lst, columns=['SDG', 'Target', 'Query', 'Not'])
df_query.to_pickle("../query/query.pkl")


In [69]:
df = pd.read_pickle("../data/dataframes/SDG/intersection_sdg_dt.pkl")
df['TXT'] = df['AB'] + " " + df['TI'] + " " + df['DE']



In [70]:
def to_raw(string):
    return fr"{string.strip()}"

In [72]:
txt = df.sample().TXT.iloc[0]
print(txt)
for ind, row in df_query.iterrows():
    if not row['Not']:
        try:
            m = re.search(to_raw(row['Query']), txt)
        except:
            print("error, weird query")
        if m:
            print(row['Target'], row['Query'])
            print(m)



Smart sustainable/sustainable smart cities, a defining context for ICT for sustainability, have recently become the leading global paradigm of urbanism. With this position, they are increasingly gaining traction and prevalence worldwide as a promising response to the mounting challenges of sustainability and the potential effects of urbanization. In the meantime, the research in this area is garnering growing attention and rapidly burgeoning, and its status is consolidating as one of the most enticing areas of investigation today. A large part of research in this area focuses on exploiting the potentials and opportunities of advanced technologies and their novel applications, especially big data computing, as an effective way to mitigate or overcome the issue of sustainable cities and smart cities being extremely fragmented as landscapes and weakly connected as approaches. In this context, one of the most appealing strands of research in the domain of smart sustainable urbanism is that

In [2]:
directory = glob.glob("../query/regex_query/*")
for path in tqdm.tqdm(directory):
    ending = path.split("/")[-1]
    new_path = "../query/regex/" + ending
    print(new_path)
    with open(path, "r", encoding="utf-8") as f:
        lst = []
        for line in f:
            reg = re.compile(r'TS = \((.+)\)(OR)?')
            m = re.search(pattern=reg, string=line)
            if m:
                line = m.group(1).strip()
                line = line.replace("\"", "").replace("*", "\w*").replace(" AND ", " ").replace("OR", "|").replace(" )", ")").replace("( ", "(").replace(" | ", "|")
                if "NOT" in line:
                    line_pos, line_neg = line.split("NOT")
                    print(line_pos)
                    line = line_pos.strip()
                if "NEAR/3" in line:
                    line_as_list = line.split("NEAR/3")
                    line_as_list = [elem.strip() for elem in line_as_list]
                    line = ""
                    for chunk in itertools.permutations(line_as_list, len(line_as_list)):
                        # print('\W+(?:\w+\W+){0,3}?'.join(chunk))
                        line = line + "|" + '\W+(?:\w+\W+){0,3}?'.join(chunk)
                    lst.append(line[1:]+"\n")
                if "NEAR/6" in line:
                    line_as_list = line.split("NEAR/3")
                    line_as_list = [elem.strip() for elem in line_as_list]
                    line = ""
                    for chunk in itertools.permutations(line_as_list, len(line_as_list)):
                        line = line + "|" + '\W+(?:\w+\W+){0,6}?'.join(chunk)
                    lst.append(line[1:]+"\n")
    # with open(new_path, 'w', encoding='utf-8') as w:
    #     print("here")
    #     w.writelines(lst)



  0%|          | 0/17 [00:00<?, ?it/s]

../query/regex/SDG-14_life_below_water.txt
here
../query/regex/SDG-17_partnerships_for_the_goals.txt
here
../query/regex/SDG-07_affordable_and_clean_energy.txt
here
../query/regex/SDG-15_life_on_land.txt
((sustainab\w*|conserv\w*|restor\w*) NEAR/3 (ecosyst\w*|terrestrial|freshwater|inland|forest\w*|rainforest\w*|agroforest\w*|deforest\w*|reforest\w*|afforest\w*|environment\w*|soil|land))  
(poaching|trafficking) (species|flora|fauna|wildlife)  
(invas\w*) NEAR/3 (alien|nonnative|non-native|nonindigenous|non-indigenous) NEAR/3 (species|animal\w*|plant\w*)  
(protect\w*) NEAR/3 (threaten\w*|endanger\w*) NEAR/3 (species|animal\w*|plant\w*)  
((prevent\w*) NEAR/3 (extinct\w*)) ((threaten\w*|endanger\w*) NEAR/3 (species|animal\w*|plant\w*))  
((ecosystem NEAR/3 biodiversity)|species diversity) (polic\w*|strateg\w*|plan|plans|planning)  
(poaching|trafficking) (species|flora|fauna|wildlife)  
here
../query/regex/SDG-04_quality_education.txt
here
../query/regex/SDG-02_zero_hunger.txt
here
../

 88%|████████▊ | 15/17 [00:32<00:04,  2.19s/it]


KeyboardInterrupt: 

In [24]:
# TEST NEAR interchangeable
directory = glob.glob("../query/regex_query/test.txt")
with open("../query/regex_query/test.txt", "r", encoding="utf-8") as f:
    lst = []
    for line in f:
        reg = re.compile(r'TS = \((.+)\)(OR)?')
        m = re.search(pattern=reg, string=line)
        if m:
            line = m.group(1).strip()
            line = line.replace("\"", "").replace("*", "\w*").replace(" AND ", " ").replace("OR", "|").replace(" )", ")").replace("( ", "(").replace(" | ", "|")
            if "NOT" in line:
                line_pos, line_neg = line.split("NOT")
                line = line_pos.strip()
            if "NEAR/3" in line:
                line_as_list = line.split("NEAR/3")
                line_as_list = [elem.strip() for elem in line_as_list]
                line_near = ""
                for chunk in itertools.permutations(line_as_list, len(line_as_list)):
                    # print('\W+(?:\w+\W+){0,3}?'.join(chunk))
                    line_near = line_near + "|" + '\W+(?:\w+\W+){0,3}?'.join(chunk)
                print(line_near[1:])
                lst.append(line_near[1:]+"\n")
with open("../query/regex_query/test_reg.txt", "w", encoding="utf-8") as f:
    f.writelines(lst)
        # .replace(' NEAR/3 ', '\W+(?:\w+\W+){0,3}?').replace(' NEAR/6 ', '\W+(?:\w+\W+){0,6}?')


(investment)\W+(?:\w+\W+){0,3}?(poverty)|(poverty)\W+(?:\w+\W+){0,3}?(investment)
(poverty)\W+(?:\w+\W+){0,3}?(eradicat\w*|reduc\w*|end|ending|alleviat\w*)|(eradicat\w*|reduc\w*|end|ending|alleviat\w*)\W+(?:\w+\W+){0,3}?(poverty)
(poverty)\W+(?:\w+\W+){0,3}?(develop\w*)\W+(?:\w+\W+){0,3}?(cooperat\w*|assistan\w*)|(poverty)\W+(?:\w+\W+){0,3}?(cooperat\w*|assistan\w*)\W+(?:\w+\W+){0,3}?(develop\w*)|(develop\w*)\W+(?:\w+\W+){0,3}?(poverty)\W+(?:\w+\W+){0,3}?(cooperat\w*|assistan\w*)|(develop\w*)\W+(?:\w+\W+){0,3}?(cooperat\w*|assistan\w*)\W+(?:\w+\W+){0,3}?(poverty)|(cooperat\w*|assistan\w*)\W+(?:\w+\W+){0,3}?(poverty)\W+(?:\w+\W+){0,3}?(develop\w*)|(cooperat\w*|assistan\w*)\W+(?:\w+\W+){0,3}?(develop\w*)\W+(?:\w+\W+){0,3}?(poverty)


In [172]:
directory = glob.glob("../query/regex_query/regex/*")
def to_raw(string):
    return fr"{string.strip()}"
def check_for_sdg(txt):
    res = []
    for path in directory:
        ending = path.split("/")[-1][:-4]
        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                reg = re.compile(to_raw(line))

                m = re.search(reg, txt)
                if m:
                    print(ending)
                    print(line)
                    print(m.group())
                    res.append((ending, m.group()))
    return res

txt = "Wireless sensor networks have been widely used in industrial IoT contexts due to their usefulsupport during the production monitoring and risk pre-warning. However, complicated and extremelyvariable industrial environments call for higher communication stability and reliability toguarantee timely transmission of monitored data. Considering mobile intelligences such as AutomatedGuided Vehicles (AGVs), within industrial scenarios, we propose a solution based on hybrid (fixedand mobile) industrial wireless sensor networks featured with a task-oriented model. The proposalincludes a heuristic modeling method adopted to assign tasks from definite orders which will be sentto the controller, and a collaborative routing algorithm exploiting the AGVs mobility on specifictrajectories. Experiments have shown that, using mobility features of the AGVs, the integratedsolution can promptly repair the network when a node and the relative link fail or the detectedquality is low, significantly reducing the energy consumption of wireless nodes and datacommunication delay, thus improving the overall throughput and reliability of industrial IoTsystems. (C) 2018 Published by Elsevier B.V."
check_for_sdg(txt)

SDG-07_affordable_and_clean_energy
((reduc\w*)\W+(?:\w+\W+){0,3}?(energy consumption))

reducing the energy consumption


[('SDG-07_affordable_and_clean_energy', 'reducing the energy consumption')]

In [25]:
def to_raw(string):
    return fr"{string.strip()}"

In [33]:
txt = "development and poverty cooperation are paramount"
with open("../query/regex_query/test_reg.txt", "r", encoding="utf-8") as f:
    for line in f:
        reg = re.compile(to_raw(line), flags=re.IGNORECASE)
        m = re.search(reg, txt)
        if m:
            print(m.group())




development and poverty cooperation


In [16]:
import re
reg = re.compile(r".*(TS.+)\)", flags=re.DOTALL)
m = re.search('(?<=abc)def', 'abcdef')
g = re.match(pattern=reg, string=txt)
g.group()

'(\n\nTS = (( "poverty" ) NEAR/3 ( "eradicat*" OR "reduc*" OR "end" OR "ending" OR "alleviat*") ) OR\n\nTS = (("poverty line*") OR ("poverty indicator*") ) OR\n\nTS = (("poverty" OR "income") NEAR/3 ("inequalit*") ) OR\n\nTS = (("poverty") NEAR/3 ("chronic*" OR "extreme") )\nOR\n\nTS = ( ( "poverty" ) NEAR/3 ( "living" OR "life" OR "child*" OR "socioeconomic*" OR "socio-economic*" OR "social welfare" OR "household*" OR "income*" ) OR ("poverty line*") )\nOR\n\nTS = ( ("social protection" OR "economic marginalization" OR "economic marginalisation" OR "poor*" OR "vulnerable") AND ("poverty" OR "income") )\nOR\n\nTS = (("access" OR "right*") NEAR/3 ("economic resource*" OR "basic service*") ) OR\n\nTS = (("ownership*") NEAR/3 ("land") )\nOR\n\nTS = (("resilien*") NEAR/3 ("poverty" OR "the poor*") ) OR\n\nTS = (("disaster*") NEAR/3 ( "number of death*") ) OR\n\nTS = (("disaster*") NEAR/3 ( "economic loss*") ) OR\n\nTS = (("disaster*") NEAR/3 ("risk reduction*") NEAR/3 ("strateg*") )\nOR\n\

In [21]:
reg = r"poverty eradication"
regg = re.compile(reg)
m = re.match(reg, "poverty eradication is paramount")

In [22]:
m.group()

'poverty eradication'

In [1]:
import pandas as pd
df_sdg = pd.read_pickle("../data/dataframes/SDG/all_sdg_fixed_dst.pkl")
df_inter = df_sdg[df_sdg['DST']]
df_dt = pd.read_pickle("../data/dataframes/digital/all_digital.pkl")

In [23]:
lst_eu = ["Austria", "Belgium", "Bulgaria", "Croatia", "Cyprus", "Czech Republic", "Denmark", "Estonia", "Finland", "France", "Germany", "Greece", "Hungary", "Ireland", "Italy",
          "Latvia", "Lithuania", "Luxembourg", "Malta", "Netherlands", "Poland",
          "Portugal", "Romania", "Slovakia", "Slovenia", "Spain", "Sweden"]

In [42]:
from collections import Counter
c_dt = Counter()
c_sdg = Counter()
c_inter = Counter()


for ind, row in df_sdg.iterrows():
    lst_country = row.CN.split(", ")
    for c in lst_country:
        c_sdg['world'] += 1
        if c in ["United States", "China"]:
            c_sdg[c] += 1
        if c in lst_eu:
            c_sdg['EU'] += 1

In [43]:
for ind, row in df_dt.iterrows():
    lst_country = row.CN.split(", ")
    for c in lst_country:
        c_dt['world'] += 1
        if c in ["United States", "China"]:
            c_dt[c] += 1
        if c in lst_eu:
            c_dt['EU'] += 1

In [44]:
for ind, row in df_inter.iterrows():
    lst_country = row.CN.split(", ")
    for c in lst_country:
        c_inter['world'] += 1
        if c in ["United States", "China"]:
            c_inter[c] += 1
        if c in lst_eu:
            c_inter['EU'] += 1

In [45]:
dict(c_sdg)

{'world': 5573338, 'EU': 1408381, 'United States': 909183, 'China': 908048}

In [46]:
c_dt

Counter({'world': 5288392,
         'China': 1358934,
         'United States': 822298,
         'EU': 1187313})

In [47]:
c_inter

Counter({'world': 211126, 'China': 44009, 'EU': 48451, 'United States': 31282})

In [50]:
df_sdg_stat = pd.DataFrame.from_dict(c_sdg, orient="index", columns=['sdg'])
df_dt_stat = pd.DataFrame.from_dict(c_dt, orient='index', columns=['dt'])
df_inter_stat = pd.DataFrame.from_dict(c_inter, orient="index", columns=['inter'])
df_sdg_stat.loc[:, 'percentage_sdg'] = df_sdg_stat.loc[:,'sdg'] / df_sdg_stat.loc['world', 'sdg']
df_dt_stat.loc[:, 'percentage_dt'] = df_dt_stat.loc[:,'dt'] / df_dt_stat.loc['world', 'dt']
df_inter_stat.loc[:, 'percentage_inter'] = df_inter_stat.loc[:,'inter'] / df_inter_stat.loc['world', 'inter']
df_stat = pd.concat([df_sdg_stat, df_dt_stat, df_inter_stat], axis=1)
df_stat

,sdg,percentage_sdg,dt,percentage_dt,inter,percentage_inter
world,5573338,1.000000,5288392,1.000000,211126,1.000000
EU,1408381,0.252700,1187313,0.224513,48451,0.229489
United States,909183,0.163131,822298,0.155491,31282,0.148167
China,908048,0.162927,1358934,0.256965,44009,0.208449


In [52]:
df_stat.to_excel('raw_stats_reports_EU.xlsx')

ModuleNotFoundError: No module named 'WordCloud'